In [8]:
import contextily as cx
import matplotlib.pyplot as plt
import cenpy
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd
from shapely import wkt
import geopandas as gpd

/Users/kathrynpanger/opt/anaconda3/envs/censusViz2/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [14]:
toxicDf = pd.read_csv("../localData/epa_tri_toxic_waste_2019.csv")
toxicDf = toxicDf.loc[toxicDf["6. CITY"] == "CHICAGO"]
toxicDf = gpd.GeoDataFrame(toxicDf, geometry=gpd.points_from_xy(toxicDf["13. LONGITUDE"], toxicDf["12. LATITUDE"])).set_crs("EPSG:4326")

In [21]:
chicagoDf = pd.read_csv("../localData/censusVars.csv")
chicagoDf['geometry'] = chicagoDf['geometry'].apply(wkt.loads)
chicagoDf = gpd.GeoDataFrame(chicagoDf, crs='epsg:4326')

In [ ]:
f, ax = plt.subplots(1,1,figsize=(20,20))
ax.set_axis_off();
chicagoDf.plot(edgecolor='slategrey', color='none', ax=ax)
cx.add_basemap(ax, crs=chicagoDf.crs.to_string(), source=cx.providers.CartoDB.Voyager, zoom=12, alpha = 1)
toxicDf.plot(ax=ax, color='yellow', alpha = 0.3, edgecolors = "black")

title = 'Location of Toxic Waste Centers\n(note: opaque markers represent more than one center at location)'
ax.set_title(title)


In [30]:
#Get correct units to make marker sizes reasonable
def categorical_to_numeric(df_column):
    unique_dict = {df_column.unique()[i]: i for i in range(len(df_column.unique()))}
    return df_column.map(unique_dict)

conversion_dict = {1:453.592, 0:1}
toxicDf["pounds_to_grams"] = categorical_to_numeric(toxicDf["47. UNIT OF MEASURE"]).map(conversion_dict)
toxicDf["adjusted_onsite_release"]=toxicDf["62. ON-SITE RELEASE TOTAL"] * toxicDf["pounds_to_grams"]

In [31]:
toxicDf["adjusted_onsite_release"]

37        3431.00
248        166.00
323          1.00
526         70.99
607          0.00
           ...   
77541    18200.00
77578        0.00
79350        2.51
79442     1709.00
79650     1314.00
Name: adjusted_onsite_release, Length: 303, dtype: float64

In [ ]:
f, ax = plt.subplots(1,1,figsize=(20,20))
ax.set_axis_off();
chicagoDf.plot(edgecolor='slategrey', color='none', ax=ax)
# # chicago_bounds.plot(edgecolor='cornsilk', color='none', ax=ax, linewidth=2)
# chicago.plot('proportion_black', cmap='Blues', ax = ax, alpha=0.5, legend = True, cax = None, color = None, vmin=0, vmax=1)
# toxicDf.plot(ax=ax, color='red', alpha = 0.5, markersize=toxicDf["62. ON-SITE RELEASE TOTAL"])
toxicDf.plot(ax=ax, color='yellow', alpha = 0.3, edgecolors = "black", markersize=toxicDf["adjusted_onsite_release"])
cx.add_basemap(ax, crs=chicagoDf.crs.to_string(), source=cx.providers.CartoDB.Voyager, zoom=12, alpha = 1)

title = 'Location of Toxic Waste Centers in Chicago: Markers Scaled By Quantity of On-Site Emissions'
ax.set_title(title)
plt.savefig("toxicMapRedux.png")

In [ ]:
f, ax = plt.subplots(1,1,figsize=(20,20))
ax.set_axis_off();
chicagoDf.plot(edgecolor='slategrey', color='none', ax=ax)
chicagoDf.plot('proportion_alone_black', cmap='Blues', ax = ax, alpha=0.5, legend = True, cax = None, color = None, vmin=0, vmax=1)

cx.add_basemap(ax, crs=chicagoDf.crs.to_string(), source=cx.providers.CartoDB.Voyager, zoom=12, alpha = 1)
toxicDf.plot(ax=ax, color='yellow', alpha = 0.3, edgecolors = "black")

title = 'Location of Toxic Waste Centers Relative to Black-Alone Population Before Considering Emission Quantity'
ax.set_title(title)
plt.savefig(f"mapImages/{title.replace(' ', '')}.png")

In [ ]:
plt.savefig(f"mapImages/{title.replace(' ', '')}.png")
f, ax = plt.subplots(1, 1, figsize=(20, 20))
ax.set_axis_off();
chicagoDf.plot(edgecolor='slategrey', color='none', ax=ax)
chicagoDf.plot('proportion_alone_black', cmap='Blues', ax=ax, alpha=0.5, legend=True, cax=None, color=None, vmin=0,
               vmax=1)

cx.add_basemap(ax, crs=chicagoDf.crs.to_string(), source=cx.providers.CartoDB.Voyager, zoom=12, alpha=1)
toxicDf.plot(ax=ax, color='yellow', alpha = 0.3, edgecolors = "black", markersize=toxicDf["adjusted_onsite_release"])

title = 'Location of Toxic Waste Centers Relative to Black-Alone Population After Considering Emission Quantity'
ax.set_title(title)
plt.savefig(f"mapImages/{title.replace(' ', '')}.png")

In [ ]:
plt.savefig(f"mapImages/{title.replace(' ', '')}.png")
f, ax = plt.subplots(1, 1, figsize=(20, 20))
ax.set_axis_off();
chicagoDf.plot(edgecolor='slategrey', color='none', ax=ax)
chicagoDf.plot('proportion_alone_white', cmap='Reds', ax=ax, alpha=0.5, legend=True, cax=None, color=None, vmin=0,
               vmax=1)

cx.add_basemap(ax, crs=chicagoDf.crs.to_string(), source=cx.providers.CartoDB.Voyager, zoom=12, alpha=1)
toxicDf.plot(ax=ax, color='yellow', alpha = 0.3, edgecolors = "black", markersize=toxicDf["adjusted_onsite_release"])

title = 'Location of Toxic Waste Centers Relative to White-Alone Population After Considering Emission Quantity'
ax.set_title(title)
plt.savefig(f"mapImages/{title.replace(' ', '')}.png")

In [91]:
quantile = chicagoDf["proportion_alone_black"].quantile(q = 0.75)
chicagoDf['pop_if_over_quantile'] = chicagoDf["proportion_alone_black"].apply(lambda x: round(x,2) if x>quantile else " ")

In [ ]:
plt.savefig(f"mapImages/{title.replace(' ', '')}.png")
f, ax = plt.subplots(1, 1, figsize=(20, 20))
ax.set_axis_off();
chicagoDf.plot(edgecolor='slategrey', color='none', ax=ax)
chicagoDf.plot('proportion_alone_black', cmap='Blues', ax=ax, alpha=0.5, legend=True, cax=None, color=None, vmin=0,
               vmax=1)

cx.add_basemap(ax, crs=chicagoDf.crs.to_string(), source=cx.providers.CartoDB.Voyager, zoom=12, alpha=1)
toxicDf.plot(ax=ax, color='yellow', alpha = 0.3, edgecolors = "black", markersize=toxicDf["adjusted_onsite_release"])
chicagoDf.apply(lambda x: ax.annotate(text=x['pop_if_over_quantile'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);#add population annotation if over quantile
title = 'Location of Toxic Waste Centers Relative to Black-Alone Population After Considering Emission Quantity \n25% Most-Black neighborhoods Labeled With Population Proportion'
ax.set_title(title)
plt.savefig(f"mapImages/{title.replace(' ', '')}.png")

In [ ]:
toxicDf